In [9]:
import numpy as np
from module import X,Y
import pandas as pd
import torch as t
import torch.nn as nn
import torch.optim as optim
import sklearn.model_selection as sk

In [19]:
X,xtest,Y,ytrain=sk.train_test_split(X,Y,test_size=0.33, random_state=42)
print(X.shape[1],Y.shape)

X_mean, X_std = X.mean(axis=0), X.std(axis=0)
X = (X - X_mean) / X_std

8 torch.Size([897, 1])


## Architructure of NN

**it is a 3 hidden layer nn with 3,4,2 neuron respectivily**

In [11]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 16),   
            nn.ReLU(),
            nn.Linear(16, 8),          
            nn.ReLU(),
            nn.Linear(8, 1)             
        )

    def forward(self, x):
        return self.net(x)


In [12]:
model = NeuralNet(X.shape[1])
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

In [13]:
for i in range(2000):
    y_pred=model(X)
    loss=criterion(y_pred,Y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1) % 200 == 0:
        with t.no_grad():
            y_class = (t.sigmoid(y_pred) >= 0.5).float()
            acc = (y_class.eq(Y).sum().item() / Y.shape[0]) * 100
        print(f"Epoch [{i+1}/{2000}] | Loss: {loss.item():.4f} | Train Acc: {acc:.2f}%")


Epoch [200/2000] | Loss: 0.3058 | Train Acc: 86.64%
Epoch [400/2000] | Loss: 0.2803 | Train Acc: 87.76%
Epoch [600/2000] | Loss: 0.2736 | Train Acc: 88.06%
Epoch [800/2000] | Loss: 0.2673 | Train Acc: 88.28%
Epoch [1000/2000] | Loss: 0.2619 | Train Acc: 88.96%
Epoch [1200/2000] | Loss: 0.2625 | Train Acc: 87.01%
Epoch [1400/2000] | Loss: 0.2548 | Train Acc: 87.61%
Epoch [1600/2000] | Loss: 0.2496 | Train Acc: 88.58%
Epoch [1800/2000] | Loss: 0.2541 | Train Acc: 87.91%
Epoch [2000/2000] | Loss: 0.2431 | Train Acc: 89.10%


In [14]:
with t.no_grad():
    y_pred_test = model(xtest)
    y_class_test = (t.sigmoid(y_pred_test) >= 0.5).float()
    acc_test = (y_class_test.eq(ytrain).sum().item() / ytrain.shape[0]) * 100
    print(f"\nTest Accuracy: {acc_test:.2f}%")


Test Accuracy: 89.85%


In [16]:
t.save(model.state_dict(), "model.pth")


In [21]:
t.save({
    "model_state": model.state_dict(),
    "mean": X_mean,
    "std": X_std
}, "model.pth")

print("Model saved successfully ✅")

Model saved successfully ✅
